In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

from joblib import Parallel, delayed
import time
from collections import defaultdict

In [3]:
# predictions_rootdir = '/home/yuncong/csd395/CSHL_patch_predictions_svm_Sat16ClassFinetuned'
# predictions_rootdir = '/home/yuncong/csd395/CSHL_patch_predictions_svm_Sat16ClassFinetuned_v2/'
predictions_rootdir = create_if_not_exists('/home/yuncong/csd395/CSHL_patch_predictions_svm_Sat16ClassFinetuned_v3')

In [4]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

labels_index = dict((j, i) for i, j in enumerate(labels))

labels_from_surround = dict( (l+'_surround', l) for l in labels[1:])

labels_surroundIncluded_list = labels[1:] + [l+'_surround' for l in labels[1:]]
labels_surroundIncluded = set(labels_surroundIncluded_list)

labels_surroundIncluded_index = dict((j, i) for i, j in enumerate(labels_surroundIncluded_list))

colors = np.random.randint(0, 255, (len(labels_index), 3))

In [5]:
stack = 'MD589'
# stack = 'MD594'
first_bs_sec, last_bs_sec = section_range_lookup[stack]
first_detection_sec, last_detection_sec = detect_bbox_range_lookup[stack]

In [9]:
n_labels = len(labels_surroundIncluded_index)
M = np.zeros((n_labels, n_labels), np.int)
M_soft = np.zeros((n_labels, n_labels))

mistakes_to_investigate = defaultdict(list)

for sec in range(first_detection_sec, last_detection_sec+1):
    
    try:
        true_labels = np.load('/home/yuncong/csd395/CSHL_patch_features_Sat16ClassFinetuned/%(stack)s/%(sec)04d/%(stack)s_%(sec)04d_roi1_labels.npy' % \
                         {'stack':stack, 'sec': sec})
    except:
        continue
    
    print sec
    
    n = len(true_labels)
    
    scores = np.zeros((n, len(labels_surroundIncluded_index)))
    for l in labels_surroundIncluded_list:
        if not l.endswith('surround'):
            scores[:, labels_surroundIncluded_index[l]] = np.load(predictions_rootdir + '/%(stack)s/%(sec)04d/%(stack)s_%(sec)04d_roi1_%(label)s_sparseScores.npy' % \
                         {'stack':stack, 'sec': sec, 'label': l})
        else:
            scores[:, labels_surroundIncluded_index[l]] = 1 - scores[:, labels_surroundIncluded_index[labels_from_surround[l]]]
        
#     hard_predictions = np.argmax(predictions, axis=1)

#     for hp, tl in zip(hard_predictions, true_labels):
#         if tl == -1: continue
#         M[tl, hp] += 1
        
    for i in range(n):
        probs = scores[i]
        tl = true_labels[i]
        if tl == -1: 
            continue
        
        true_name = labels_surroundIncluded_list[tl]
        correct_prob = probs[labels_surroundIncluded_index[true_name]]
        if correct_prob < 1e-1:
#             print i, true_name, correct_prob
            
            mistakes_to_investigate[sec].append((i, true_name, probs))
            
mistakes_to_investigate.default_factory = None

150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316


In [60]:
true_labels = np.load('/home/yuncong/csd395/CSHL_patch_features_Sat16ClassFinetuned/%(stack)s/%(sec)04d/%(stack)s_%(sec)04d_roi1_labels.npy' % \
                         {'stack':stack, 'sec': 160})
mapping_indices_to_label = dict(zip(indices_allROIs_allSections[160]['roi1'], 
             [labels_surroundIncluded_list[tl] if tl != -1 else None for tl in true_labels]))

In [61]:
mapping_indices_to_label[41155]

'sp5_surround'

In [62]:
sample_locations[41155]

array([11256,  9128])

In [13]:
# Load sample locations

patches_rootdir = '/home/yuncong/CSHL_data_patches/'

import pandas as pd

table_filepath = os.path.join(patches_rootdir, '%(stack)s_indices_allROIs_allSections.h5'%{'stack':stack})
indices_allROIs_allSections = pd.read_hdf(table_filepath, 'indices_allROIs_allSections')
grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')

patch_size, stride, w, h = grid_parameters.tolist()
half_size = patch_size/2
ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                 indexing='xy')
sample_locations = np.c_[xs.flat, ys.flat]

In [57]:
def mark_mistakes(stack, sec, mistakes_to_investigate, sample_locations, half_size):
    
    indices = [indices_allROIs_allSections[sec]['roi1'][m[0]] for m in mistakes_to_investigate[sec]]

    titles = ['%d %s %.3f' % (indices_allROIs_allSections[sec]['roi1'][m[0]], m[1], m[2][labels_surroundIncluded_index[m[1]]]) for m in mistakes_to_investigate[sec]]
    
    patches = []
    
    dm = DataManager(stack=stack)
    
    img_fn = dm._get_image_filepath(stack=stack, section=sec, version='rgb-jpg')
    img = imread(img_fn)
    
    h, w = img.shape[:2]
    
    for ind, title in zip(indices, titles):
        xc, yc = sample_locations[ind]
    
        patch_ymin = max(0, yc-half_size)
        patch_ymax = min(h, yc+half_size)
        patch_xmin = max(0, xc-half_size)
        patch_xmax = min(w, xc+half_size)
    
        cv2.rectangle(img, 
                      (patch_xmin, patch_ymin),
                      (patch_xmax, patch_ymax),
                      (255, 0, 0), 5)
        
        cv2.putText(img, title, (patch_xmin, patch_ymin-10), cv2.FONT_HERSHEY_DUPLEX, 2, ((255,0,0)), 1)
        
    return img

In [58]:
mistake_img = mark_mistakes(stack, 160, mistakes_to_investigate, sample_locations, half_size)
display_image(mistake_img)

/oasis/projects/nsf/csd395/yuncong/Brain/learning/tmp.jpg

In [ ]:
def patches_from_indices(stack, sec, indices, sample_locations, half_size, return_context=False, context_size=500):
    
    patches = []
    context_images = []
    
    dm = DataManager(stack=stack)
    
    img_fn = dm._get_image_filepath(stack=stack, section=sec, version='rgb-jpg')
    img = imread(img_fn)
    
    h, w = img.shape[:2]
    
    for i in indices:
        xc, yc = sample_locations[i]
    
        patch_ymin = max(0, yc-half_size)
        patch_ymax = min(h, yc+half_size)
        patch_xmin = max(0, xc-half_size)
        patch_xmax = min(w, xc+half_size)
    
#         print patch_xmin, patch_xmax, patch_ymin, patch_ymax
    
        patch = img[patch_ymin:patch_ymax+1, patch_xmin:patch_xmax+1]
        patches.append(patch)
        
        if return_context:
            context_ymin = max(0, yc-context_size)
            context_ymax = min(h, yc+context_size)
            context_xmin = max(0, xc-context_size)
            context_xmax = min(w, xc+context_size)
            
#             print context_xmin, context_xmax, context_ymin, context_ymax
        
            context_image = img[context_ymin:context_ymax+1, context_xmin:context_xmax+1].copy()
            cv2.rectangle(context_image, 
                          (patch_xmin - context_xmin, patch_ymin - context_ymin),
                          (patch_xmax - context_xmin, patch_ymax - context_ymin),
                        (255, 0, 0), 5)      
        
#             rs, cs = polygon_perimeter([patch_ymin - context_ymin, patch_ymin - context_ymin, 
#                                         patch_ymax - context_ymin, patch_ymax - context_ymin], 
#                                        [patch_xmin - context_xmin, patch_xmax - context_xmin, 
#                                         patch_xmax - context_xmin, patch_xmin - context_xmin],
#                                       shape=context_image.shape)
#             context_image[rs, cs] = (255,0,0)
            
            context_images.append(context_image)
            
    if return_context:
        return patches, context_images
    else:
        return patches

In [ ]:
sec = first_detection_sec
indices_to_investigate = [indices_allROIs_allSections[sec]['roi1'][m[0]] for m in mistakes_to_investigate[sec]]
patches, context_images = patches_from_indices(stack, first_detection_sec, indices_to_investigate, sample_locations, half_size,
                                              return_context=True, context_size=1000)

In [ ]:
titles = ['%s %.3f' % (m[1], m[2][labels_surroundIncluded_index[m[1]]]) for m in mistakes_to_investigate[sec]]

In [ ]:
plt.figure(figsize=(10,10));
plt.imshow(context_images[1]);

In [ ]:
display_images_in_grids(patches, 5, titles=titles)